In [11]:
import unidecode
import string
import random
import re
import torch
import torch.nn as nn
from torch.autograd import Variable
import time, math

In [22]:
all_characters = string.printable
n_characters = len(all_characters)
#print("Character length...",char_len)
file = unidecode.unidecode(open("corpus.txt").read())
file_len = len(file)
print(len_file)

10761287


In [23]:
chunk_len = 200

def random_chunk():
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]

print(random_chunk())

ty so far is as usual for a typical Thursday.
Would you like to do some more exercises with me?
What would you like to do next?
"We can talk, relax with mindfulness, or do something physical."
"Hi! I'm


In [24]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.encoder = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
    
    def forward(self, input, hidden):
        input = self.encoder(input.view(1, -1))
        output, hidden = self.gru(input.view(1, 1, -1), hidden)
        output = self.decoder(output.view(1, -1))
        return output, hidden

    def init_hidden(self):
        return Variable(torch.zeros(self.n_layers, 1, self.hidden_size))
        

In [25]:
# Turn string into list of longs
def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = all_characters.index(string[c])
    return Variable(tensor)

print(char_tensor('abcDEF'))

Variable containing:
 10
 11
 12
 39
 40
 41
[torch.LongTensor of size 6]



In [26]:
def random_training_set():    
    chunk = createChunk()
    inp = char_tensor(chunk[:-1])
    target = char_tensor(chunk[1:])
    return inp, target

In [27]:
def evaluate(prime_str='A', predict_len=100, temperature=0.8):
    hidden = decoder.init_hidden()
    prime_input = char_tensor(prime_str)
    predicted = prime_str

    # Use priming string to "build up" hidden state
    for p in range(len(prime_str) - 1):
        _, hidden = decoder(prime_input[p], hidden)
    inp = prime_input[-1]
    
    for p in range(predict_len):
        output, hidden = decoder(inp, hidden)
        
        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        
        # Add predicted character to string and use as next input
        predicted_char = all_characters[top_i]
        predicted += predicted_char
        inp = char_tensor(predicted_char)

    return predicted

In [28]:
def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [29]:
def train(inp, target):
    hidden = decoder.init_hidden()
    decoder.zero_grad()
    loss = 0

    for c in range(chunk_len):
        output, hidden = decoder(inp[c], hidden)
        loss += criterion(output, target[c])

    loss.backward()
    decoder_optimizer.step()

    return loss.data[0] / chunk_len

In [ ]:
n_epochs = 5000
print_every = 100
plot_every = 10
hidden_size = 500
n_layers = 1
lr = 0.005

decoder = RNN(n_characters, hidden_size, n_characters, n_layers)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

start = time.time()
all_losses = []
loss_avg = 0

for epoch in range(1, n_epochs + 1):
    loss = train(*random_training_set())       
    loss_avg += loss

    if epoch % print_every == 0:
        print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs * 100, loss))
        print(evaluate('Wh', 100), '\n')

    if epoch % plot_every == 0:
        all_losses.append(loss_avg / plot_every)
        loss_avg = 0

 38s (100 2%) 2.0653]
Whatd you #BRY.
For beelit."
Rexpreath whese whind you carve to in you feeling or #BRY.gi
Feell where  

 16s (200 4%) 0.9017]
Wheren usee for -> you  and wat makas bay Helk
Ok
"Ye you're for nam the for frit the for  -> Say feel 

 4s (300 6%) 1.3663]
What gratitude and how se you're ned or help unde st or ing not you bak and them por ind feeling arst. 

 54s (400 8%) 2.0323]
Whalk activations see would you thanks that though the theful to thing you to what any say #CBT you ta 

 11s (500 10%) 1.6926]
Wh though that active."
"Tis alalongh its is is inse from to fiend, help you day. Aned to to the food  

 58s (600 12%) 1.4138]
Whenegatich and will usee to sees or wise you like you sarnt alaart an with a sidee your hacke of to w 

 40s (700 14%) 1.0461]
What's your name?
Here feeling bad and can low?
I can a ram You are are from lise realillog, as better 

 23s (800 16%) 1.6447]
What you reathe feel. And wis thoughte to the far. Whear stress of everyday life. When